# Find proteins within the 8p loss event (cis) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "q"
TRANS_OR_CIS = "cis"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,lscc_Database_ID,luad_Database_ID,brca_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
ANKRD46,NP_001257308.1|NP_001257306.1,NP_001257308.1|NP_001257306.1,NP_001257308.1|NP_001257307.1,NP_001257307,1.035462e-01,NaN,0.902699,0.159644,1.916761e-01,0.145248,0.503595,NaN,0.035207,0.393443,0.479780,0.463378
ASAP1,NP_060952.2|NP_001234925.1|NP_001349854.1|NP_0...,NP_060952.2|NP_001234925.1|NP_001349854.1|NP_0...,NP_060952.2|NP_001234925.1,NP_001234925,1.769220e-02,6.903566e-01,0.694614,0.980541,6.583929e-01,0.159556,0.333673,0.050545,-0.031039,0.010762,-0.097382,0.104059
ATAD2,NP_054828.2,NP_054828.2|NP_001341036.1|NP_060022.2|NP_0012...,NP_054828.2,NP_054828,8.600138e-08,1.917050e-01,0.726257,0.021111,1.143549e-03,0.748641,1.647994,0.206485,0.058610,0.872430,1.113816,0.073793
ATP6V1C1,NP_001686.1,NP_001686.1,NP_001686.1,NP_001686,1.024787e-03,5.183022e-01,0.734683,0.565390,6.583929e-01,0.019453,0.489741,0.037578,0.025330,-0.116081,0.100026,0.253735
C8orf37,NP_808880.1|NP_001350189.1,NP_808880.1|NP_001350189.1,NP_808880.1,NP_808880,1.104241e-01,NaN,0.988925,0.021111,1.608260e-01,0.335792,0.389962,NaN,-0.001730,0.649286,0.429484,0.207518
CA1,NP_001122301.1|NP_001278896.1,NP_001278897.1,NP_001158302.1|NP_001278896.1,NP_001158302,1.221581e-01,4.275252e-01,0.652917,0.574403,5.729770e-01,0.980384,0.880698,-0.207660,0.107769,0.287876,-0.451060,0.023665
CA1,NP_001122301.1|NP_001278896.1,NP_001122301.1|NP_001278896.1,NP_001158302.1|NP_001278896.1,NP_001158302,1.221581e-01,4.275252e-01,0.652917,0.574403,6.178879e-01,0.980384,0.880698,-0.207660,0.107769,0.287876,-0.270333,0.023665
CA1,NP_001278897.1,NP_001278897.1,NP_001158302.1|NP_001278896.1,NP_001158302,1.221581e-01,4.275252e-01,0.652917,0.980541,5.729770e-01,0.980384,0.880698,-0.207660,0.107769,0.027495,-0.451060,0.023665
CA1,NP_001278897.1,NP_001122301.1|NP_001278896.1,NP_001158302.1|NP_001278896.1,NP_001158302,1.221581e-01,4.275252e-01,0.652917,0.980541,6.178879e-01,0.980384,0.880698,-0.207660,0.107769,0.027495,-0.270333,0.023665


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,ANKRD46,NP_001257308.1|NP_001257307.1,1.035462e-01,0.503595
1,brca,ASAP1,NP_060952.2|NP_001234925.1,1.769220e-02,0.333673
2,brca,ATAD2,NP_054828.2,8.600138e-08,1.647994
3,brca,ATP6V1C1,NP_001686.1,1.024787e-03,0.489741
4,brca,C8orf37,NP_808880.1,1.104241e-01,0.389962
5,brca,CA13,NP_940986.1,2.280200e-01,0.426867
6,brca,CA3,NP_005172.1,1.726618e-01,0.801611
7,brca,CHMP4C,NP_689497.1,1.941474e-01,0.362290
8,brca,CNGB3,NP_061971.3,8.500956e-02,-0.621735
9,brca,COL14A1,NP_066933.1,1.840865e-03,-1.627590


## Select the proteins with a significant change, and take a detour to make some plots

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = long_results[long_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title=f"Chr {CHROMOSOME}{ARM} {TRANS_OR_CIS} effects"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [7]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [8]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,brca,ASAP1,NP_060952.2|NP_001234925.1,1.769220e-02,0.333673,1
1,brca,ATAD2,NP_054828.2,8.600138e-08,1.647994,1
2,brca,ATP6V1C1,NP_001686.1,1.024787e-03,0.489741,1
3,brca,COL14A1,NP_066933.1,1.840865e-03,-1.627590,-1
4,brca,CPNE3,NP_003900.1|NP_055242.1|NP_705900.1,1.100670e-02,0.418513,1
5,brca,CSMD3,NP_937756.1|NP_937757.1|NP_443132.3,6.655560e-03,0.713232,1
6,brca,DCAF13,NP_056235.4,5.423511e-06,0.754006,1
7,brca,DECR1,NP_001317504.1|NP_001350.1,3.830865e-04,0.767432,1
8,brca,DPYS,NP_001376.1,2.280105e-02,-0.941788,-1
9,brca,DSCC1,NP_076999.2,4.922376e-05,1.091734,1


In [9]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
CPNE3,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
OTUD6B,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
POP1,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
RMDN1,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
YWHAZ,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
SQLE,"[brca, colon, hnscc, lscc, luad]",1,5
RIDA,"[brca, colon, lscc, luad, ovarian]",1,5
STK3,"[brca, colon, lscc, luad, ovarian]",1,5
MTERF3,"[brca, colon, hnscc, luad]",1,4


In [10]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

6     5
5     3
4    10
3    15
2    14
1    23
Name: num_cancers, dtype: int64

## Save results

In [11]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")